## Problem Set #7

MACS 30100, Dr. Evans 

Shuting Chen 

## 1. Classifier "horse" race 

In [85]:
# Import packages 
import numpy as np 
import pandas as pd
from sklearn.linear_model import LogisticRegression

### Part (a).

In [86]:
# Import data 
auto = pd.read_csv("Auto.csv", na_values = '?')
auto.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      5
weight          0
acceleration    0
year            0
origin          0
name            0
dtype: int64

In [87]:
auto = auto.dropna()
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [88]:
# Create a binary variable mpg_high
median_mpg = auto.mpg.median()
auto['mpg_high'] = auto.mpg.apply(lambda x: 1 if x>= median_mpg else 0)
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,mpg_high
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,0
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,0
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,0
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,0
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,0


In [89]:
auto['const'] = 1
X = pd.concat((auto.const, auto.loc[:,'cylinders':'origin']), axis = 1)
y = auto.mpg_high 
Xvals = X.values
yvals = y.values 

In [90]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
k = 4
kf = KFold(n_splits=k, shuffle=True, random_state=15)
kf.get_n_splits(Xvals)

MSE_vec_kf = np.zeros(k)
e_0 = np.zeros(k)
e_1 = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(Xvals):
    print('k index=', k_ind)
    X_train, X_test = Xvals[train_index], Xvals[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    LogReg = LogisticRegression()
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    MSE_vec_kf[k_ind] = ((y_test - y_pred) ** 2).mean()
    err0 = ((y_pred == 0)*(np.abs(y_test - y_pred))).sum() / (y_pred == 0).sum()
    e_0[k_ind] = (err0)
    err1 = ((y_pred == 1)*(np.abs(y_test - y_pred))).sum() / (y_pred == 1).sum()
    e_1[k_ind] = (err1)
    print('MSE for test set', k_ind, 'is', MSE_vec_kf[k_ind])
    print('Error rate (mpg_high, 0): ', err0)
    print('Error rate (mpg_high, 1): ', err1)
    print('\n', classification_report(y_test, y_pred, digits=3))
    k_ind += 1

MSE_kf = MSE_vec_kf.mean()
print('The MSE of the model =', MSE_kf)
print('The average error rate for mpg_high = 0:', e_0.mean())
print('The average error rate for mpg_high = 1:', e_1.mean())

k index= 0
MSE for test set 0 is 0.0918367346939
Error rate (mpg_high, 0):  0.0576923076923
Error rate (mpg_high, 1):  0.130434782609

              precision    recall  f1-score   support

          0      0.942     0.891     0.916        55
          1      0.870     0.930     0.899        43

avg / total      0.910     0.908     0.908        98

k index= 1
MSE for test set 1 is 0.102040816327
Error rate (mpg_high, 0):  0.122448979592
Error rate (mpg_high, 1):  0.0816326530612

              precision    recall  f1-score   support

          0      0.878     0.915     0.896        47
          1      0.918     0.882     0.900        51

avg / total      0.899     0.898     0.898        98

k index= 2
MSE for test set 2 is 0.132653061224
Error rate (mpg_high, 0):  0.152173913043
Error rate (mpg_high, 1):  0.115384615385

              precision    recall  f1-score   support

          0      0.848     0.867     0.857        45
          1      0.885     0.868     0.876        53

avg 

### Part (b). 

In [91]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
auto_rf = RandomForestClassifier(n_estimators=20, max_features=2, bootstrap=True,
                                  oob_score=True, random_state=25)
X1 = auto.loc[:,'cylinders':'origin']
X1vals = X1.values
auto_rf.fit(X1vals, yvals)
y_pred_df = pd.DataFrame({'y':yvals, 'ypred': auto_rf.oob_decision_function_.T[1]})
y_pred_df.ypred = y_pred_df.ypred.apply(lambda x: 1 if x >= 0.5 else 0)
MSE_rf = mean_squared_error(yvals, y_pred_df.ypred)

y_pred_df_0 = y_pred_df[y_pred_df.ypred < 0.5] 
MES_rf_0 = mean_squared_error(y_pred_df_0.y, y_pred_df_0.ypred)

y_pred_df_1 = y_pred_df[y_pred_df.ypred >= 0.5] 
MES_rf_1 = mean_squared_error(y_pred_df_1.y, y_pred_df_1.ypred)

In [92]:
print('MSE for the random forest model =', MSE_rf)
print('MSE for mpg_high = 0:', MES_rf_0)
print('MSE for mpg_high = 1:', MES_rf_1)

MSE for the random forest model = 0.0714285714286
MSE for mpg_high = 0: 0.0578947368421
MSE for mpg_high = 1: 0.0841584158416


### Part (c). 

In [93]:
from sklearn import svm
k = 4
kf_svm = KFold(n_splits=k, shuffle=True, random_state=15)
kf_svm.get_n_splits(Xvals)

MSE_vec_kf = np.zeros(k)
e_svm_0 = np.zeros(k)
e_svm_1 = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(Xvals):
    print('k index=', k_ind)
    X_train, X_test = Xvals[train_index], Xvals[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    svc = svm.SVC(kernel='rbf', gamma = 0.2, C=1)
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    MSE_vec_kf[k_ind] = ((y_test - y_pred) ** 2).mean()
    err0 = ((y_pred == 0)*(np.abs(y_test - y_pred))).sum() / (y_pred == 0).sum()
    e_svm_0[k_ind] = (err0)
    err1 = ((y_pred == 1)*(np.abs(y_test - y_pred))).sum() / (y_pred == 1).sum()
    e_svm_1[k_ind] = (err1)
    print('MSE for test set', k_ind, 'is', MSE_vec_kf[k_ind])
    print('Error rate (mpg_high, 0): ', err0)
    print('Error rate (mpg_high, 1): ', err1)
    print('\n',classification_report(y_test, y_pred, digits=3))
    k_ind += 1

MSE_kf = MSE_vec_kf.mean()
print('The MSE of the model =', MSE_kf)
print('The average error rate for mpg_high = 0:', e_svm_0[~np.isnan(e_svm_0)].mean())
print('The average error rate for mpg_high = 1:', e_svm_1[[~np.isnan(e_svm_1)]].mean())

k index= 0
MSE for test set 0 is 0.540816326531
Error rate (mpg_high, 0):  0.0
Error rate (mpg_high, 1):  0.552083333333

              precision    recall  f1-score   support

          0      1.000     0.036     0.070        55
          1      0.448     1.000     0.619        43

avg / total      0.758     0.459     0.311        98

k index= 1
MSE for test set 1 is 0.520408163265
Error rate (mpg_high, 0):  0.520408163265
Error rate (mpg_high, 1):  nan

              precision    recall  f1-score   support

          0      0.480     1.000     0.648        47
          1      0.000     0.000     0.000        51

avg / total      0.230     0.480     0.311        98

k index= 2
MSE for test set 2 is 0.520408163265
Error rate (mpg_high, 0):  0.53125
Error rate (mpg_high, 1):  0.0

              precision    recall  f1-score   support

          0      0.469     1.000     0.638        45
          1      1.000     0.038     0.073        53

avg / total      0.756     0.480     0.332     

/Users/shuting/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in long_scalars
/Users/shuting/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Part (d). 

According to the results shown above, Random Forest model is the best one for prediction of 'mpg_high'. One reason is that Random Forest model has the smallest MSE of 0.0714, while the logistic model with 4-fold corss validation has MSE of 0.1071 and SVM model has the largest MSE of 0.5077. Besides, Random Forest model provides lowest error rates for both categories of 'mpg_high', while logistic model has smaller error rates than SVM model for each category of 'mpg_high'.